# Creating Azure Resources



In [ ]:
from azure.mgmt.resource import ResourceManagementClient
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource.resources.models import DeploymentMode
import json

Start with specifying your subscription and resource group information and log in.

In [ ]:
# Subscription & resource group
SUBSCRIPTION_NAME = 'Team Danielle Internal'
SUBSCRIPTION_ID = 'edf507a2-6235-46c5-b560-fd463ba2e771'
RESOURCE_GROUP = 'baimm'
LOCATION = 'eastus'

In [ ]:
# Login to subscription
!az login

In [ ]:
# Select subscription
!az account set -s "{SUBSCRIPTION_NAME}"

In [ ]:
# Create resource group (skip if you are using an existing resource group)
!az group create -l {LOCATION} -n {RESOURCE_GROUP}

You will also need service principal credentials for authentication. Enter your service principal credentials below. If you need to create new credentials, follow the instructions [here](https://docs.microsoft.com/en-us/cli/azure/create-an-azure-service-principal-azure-cli?view=azure-cli-latest), or simply run the following and record the credentials below:

```az ad sp create-for-rbac```

In [ ]:
# Service principal info 
SP_CLIENT = '095d4aa7-a8d5-4f24-aa96-c631e3dee97c'
SP_SECRET = 'a9b478dc-dbdb-403f-bd6d-1dee6c0b1bd4'
SP_TENANT = '72f988bf-86f1-41af-91ab-2d7cd011db47'

The following are parameters needed to create and access the main Azure resources. These include: Azure Container Registry (ACR), Batch AI, Blob Storage, and Logic Apps parameters. 

You can use the default values below as is.

In [ ]:
# ACR
ACR_NAME = 'baimmacr'
ACR_SERVER = 'baimmacr.azurecr.io'
BAI_DOCKER_IMG = 'baimmacr.azurecr.io/baimmimg:v1'
SCHED_DOCKER_IMG = 'baimmacr.azurecr.io/baimmschedimg:v1'

# Batch AI
BAI_CLUSTER_NAME = 'baimmcluster'
BAI_WORKSPACE = 'baimmws'
BAI_USER = 'baimmuser'
BAI_PASS = 'baimmpass'
BAI_VM_SIZE = 'Standard_D2'
BAI_VM_IMG = 'UbuntuLTS'
BAI_NODES_MIN = 0
BAI_NODES_MAX = 3

# Blob storage
BFS_CONTAINER = 'bfs' # shared across Batch AI nodes under /mnt/batch/tasks/shared/LS_root/mounts/bfs
BLOB_ACCOUNT = 'baimmstorage'
MODELS_CONTAINER = 'models'
PREDS_CONTAINER = 'preds'
DATA_CONTAINER = 'data'

# Logic App
LA_ACI_CON = 'aci'
LA_WORKFLOW = 'baimmscheduler'
LA_ACI_CON_JSON = 'sched/api_con_template.json'
LA_JSON = 'sched/logic_app_template.json'
LA_ACI_CONTAINER_NAME = 'baimmschedcontainer'
LA_ACI_CONTAINER_GROUP = 'baimmcontainergroup'

In [ ]:
# Create ACR
!az acr create --resource-group {RESOURCE_GROUP} --name {ACR_NAME} --sku Basic

In [ ]:
# Create Blob storage account
!az storage account create -n {BLOB_ACCOUNT} -g {RESOURCE_GROUP} -l {LOCATION}

In [ ]:
# Retrieve Blob storage key
blob_keys = !az storage account keys list -g {RESOURCE_GROUP} -n {BLOB_ACCOUNT} --output tsv
blob_key = blob_keys[0].split('\t')[2]

In [ ]:
# Create models, predictions and data containers
!az storage container create -n {MODELS_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}
!az storage container create -n {PREDS_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}
!az storage container create -n {DATA_CONTAINER} --account-key {blob_key} --account-name {BLOB_ACCOUNT}

In [ ]:
# Create Batch AI cluster and set auto-scaling
!az batchai workspace create -g {RESOURCE_GROUP} -n {BAI_WORKSPACE}
!az batchai cluster create -g {RESOURCE_GROUP} -n {BAI_CLUSTER_NAME} -w {BAI_WORKSPACE} -s {BAI_VM_SIZE} -i {BAI_VM_IMG} --min {BAI_NODES_MIN} --max {BAI_NODES_MAX} -u {BAI_USER} -p {BAI_PASS} --storage-account-name {BLOB_ACCOUNT} --storage-account-key {blob_key} --bfs-name {BFS_CONTAINER}
!az batchai cluster auto-scale -g {RESOURCE_GROUP} -w {BAI_WORKSPACE} -n {BAI_CLUSTER_NAME} --min {BAI_NODES_MIN} --max {BAI_NODES_MAX}

In [ ]:
# Create config file for scoring script (predict.py)
score_config = {"blob_account": BLOB_ACCOUNT,
                "blob_key": blob_key,
                "models_blob_container": MODELS_CONTAINER,
                "data_blob_container": DATA_CONTAINER,
                "data_blob" : DATA_BLOB,
                "predictions_blob_container": PREDS_CONTAINER}

with open('batchai/predict_config.json', 'w') as f:
    json.dump(score_config, f, indent=4)


In [ ]:
# Create Batch AI docker img
!docker build -f batchai/Dockerfile -t {BAI_DOCKER_IMG} .

In [ ]:
# Validate that the image was created
!docker images

In [ ]:
# Login to ACR and push docker image
!az acr login --name {ACR_NAME}
!docker push {BAI_DOCKER_IMG}

In [ ]:
# Copy models from local dir to blob container
!az storage blob upload-batch -d {MODELS_CONTAINER} -s models --account-name {BLOB_ACCOUNT}

In [ ]:
# Copy dataset to blob
!az storage blob upload -c {DATA_CONTAINER} -f data/'sensor_data.csv' -n 'sensor_data.csv' --account-name {BLOB_ACCOUNT}

In [ ]:
# Enable ACR admin account authentication
!az acr update -n {ACR_NAME} --admin-enabled true

In [ ]:
# Get ACR's password (user is {ACR_NAME})
acr_cred = !az acr credential show --name {ACR_NAME} 
acr_password = acr_cred[4].split(':')[1].replace('"','').replace(' ','')

In [ ]:
# Create config file for Batch AI job submission script (submit_jobs.py)
submit_jobs_config = {
  "sp_tenant": SP_TENANT,
  "sp_client": SP_CLIENT,
  "sp_secret": SP_SECRET,
  "resource_group_name": RESOURCE_GROUP,
  "subscription_id": SUBSCRIPTION_ID,
  "work_space": BAI_WORKSPACE,
  "experiment_name": "baimm_score",
  "cluster_name": BAI_CLUSTER_NAME,
  "location": LOCATION,
  "acr_server": ACR_SERVER,
  "acr_image": BAI_DOCKER_IMG,
  "acr_user": ACR_NAME,
  "acr_password": acr_password,
  "command_line": "python predict.py {0} {1} {2}",
  "std_out_err_path_prefix": '/mnt/batch/tasks/shared/LS_root/mounts/{}'.format(BFS_CONTAINER),
  "config_file_path": "predict_config.json",
  "node_count": 2,
  "device_ids": [ 1, 2, 3 ],
  "tags": [ 1, 2, 3, 4, 5 ],
  "job_name": "baimm_predict{0}_{1}" # job name template
}

with open('sched/bai_pred_config.json', 'w') as f:
    json.dump(submit_jobs_config, f, indent=4)

In [ ]:
# Create scheduling docker img
!docker build -f sched/Dockerfile -t {SCHED_DOCKER_IMG} .

In [ ]:
# Login to ACR and push docker image
!az acr login --name {ACR_NAME}
!docker push {SCHED_DOCKER_IMG}

In [ ]:
# Create credentials and ARM client
credentials = ServicePrincipalCredentials(client_id=SP_CLIENT,
                                          secret=SP_SECRET,
                                          tenant=SP_TENANT)
arm_client = ResourceManagementClient(credentials, SUBSCRIPTION_ID)

In [40]:
# Create an ACI API connection
with open('sched/api_con_template.json') as f:
    aci_api_con_template = json.load(f)

aci_api_con_params = {"location": {"value": LOCATION},
                      "name": {"value": LA_ACI_CON},
                      "subscription_id": {"value": SUBSCRIPTION_ID}
                      }

aci_api_con_props = {
    'mode': DeploymentMode.incremental,
    'template': aci_api_con_template,
    'parameters': aci_api_con_params
}

arm_client.deployments.create_or_update(RESOURCE_GROUP, LA_ACI_CON, aci_api_con_props)


In [41]:
# Create Logic App
with open('sched/logic_app_template.json') as f:
    logic_app_template = json.load(f)

logic_app_params = {"location": {"value": LOCATION},
                    "resource_group": { "value": RESOURCE_GROUP },
                    "name": {"value": LA_WORKFLOW},
                    "subscription_id": {"value": SUBSCRIPTION_ID},
                    "container_name": { "value": LA_ACI_CONTAINER_NAME },
                    "container_group": { "value": LA_ACI_CONTAINER_GROUP },
                    "image_name": { "value": SCHED_DOCKER_IMG },
                    "acr_pass": { "value": acr_password },
                    "acr_user": { "value": ACR_NAME },
                    "acr_server": { "value": ACR_SERVER },
                    "aci_connection_name": { "value": LA_ACI_CON }
                    }

logic_app_props = {
    'mode': DeploymentMode.incremental,
    'template': logic_app_template,
    'parameters': logic_app_params
}

arm_client.deployments.create_or_update(RESOURCE_GROUP, LA_WORKFLOW, logic_app_props)


In [ ]:
go to portal 
authenticate 
run



In [ ]:
logic_app_template

In [ ]:
acr_password
